JDC: I've been printing the work accumulated over the first 5000 steps with the default K_RMSD compared with K_RMSD=0 and only changing one set of lambda_* parameters at a time.
It appears that changing the bond lambdas causes the biggest difference in work accumulation between with and without restraints

Script: /home/chodera/equilibrated_restraint_experiments/281/test_neq.py

In [1]:
import openmm
from openmm import unit

# prefix = '/data/chodera/zhangi/perses_benchmark/neq/15/281/'
prefix = '/home/chodera/equilibrated_restraint_experiments/281/'

def read(filename):
    print(f'Reading {filename}...')
    with open(filename, 'r') as infile:
        return openmm.XmlSerializer.deserialize(infile.read())
    
def test(lambda_param=None):
    
    system = read(f'{prefix}281_apo_system.xml')
    state = read(f'{prefix}281_apo_state.xml')
    integrator = read(f'{prefix}integrator.xml')

    forces = { force.getName() : force for (index, force) in enumerate(system.getForces()) }
    cvforce = forces['CustomCVForce']

    platform = openmm.Platform.getPlatformByName('CUDA')
    platform.setPropertyDefaultValue('Precision', 'mixed')
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
    
    print("************KMSD != 0************")
    print('Creating context...')

    context = openmm.Context(system, integrator, platform)
    context.setState(state)

    print('Integrating...')
    niterations = 5
    nsteps_per_iteration = 1000
    for iteration in range(niterations):
        
        if lambda_param is not None: # Only allow one set of lambdas to be changing
            print(f"Changing only one lambda: {lambda_param}")
            for k, v in context.getParameters().items():
                
                if 'old' in k:
                    if lambda_param in k:
                        pass
                    else:
                        context.setParameter(k, 1.0)
                elif 'new' in k or 'reciprocal' in k:
                    if lambda_param in k:
                        pass
                    else:
                        context.setParameter(k, 0)
            
        step = integrator.getGlobalVariableByName('step')
        lambda_ = integrator.getGlobalVariableByName('lambda')
        protocol_work = integrator.getGlobalVariableByName('protocol_work')
        time = context.getTime()
        cvs = cvforce.getCollectiveVariableValues(context)
        rmsd = cvs[0]
        print(f'step {step:10.0f} : time {time/unit.nanoseconds:10.3f} ns | lambda {lambda_:16.12f} | protocol_work = {protocol_work:12.3f} | RMSD = {rmsd:12.5f} nm')
#         for k, v in context.getParameters().items():
#             print(k, v)
        integrator.step(nsteps_per_iteration)

    del context, integrator
    
    print("************K_RMSD == 0************")
    print('Creating context...')

    integrator = read(f'{prefix}integrator.xml')
    
    context = openmm.Context(system, integrator, platform)
    context.setState(state)
    
    print(f'Setting K_RMSD to 0')
    context.setParameter('K_RMSD', 0.0)

    print('Integrating...')
    niterations = 5
    nsteps_per_iteration = 1000
    for iteration in range(niterations):
        
        if lambda_param is not None: # Only allow one set of lambdas to be changing
            print(f"Changing only one lambda: {lambda_param}")
            for k, v in context.getParameters().items():
                
                if 'old' in k:
                    if lambda_param in k:
                        pass
                    else:
                        context.setParameter(k, 1.0)
                elif 'new' in k or 'reciprocal' in k:
                    if lambda_param in k:
                        pass
                    else:
                        context.setParameter(k, 0)
            
        step = integrator.getGlobalVariableByName('step')
        lambda_ = integrator.getGlobalVariableByName('lambda')
        protocol_work = integrator.getGlobalVariableByName('protocol_work')
        time = context.getTime()
        cvs = cvforce.getCollectiveVariableValues(context)
        rmsd = cvs[0]
        print(f'step {step:10.0f} : time {time/unit.nanoseconds:10.3f} ns | lambda {lambda_:16.12f} | protocol_work = {protocol_work:12.3f} | RMSD = {rmsd:12.5f} nm')
        integrator.step(nsteps_per_iteration)
    
    del context, integrator



## Run neq

In [2]:
test()

Reading /home/chodera/equilibrated_restraint_experiments/281/281_apo_system.xml...
Reading /home/chodera/equilibrated_restraint_experiments/281/281_apo_state.xml...
Reading /home/chodera/equilibrated_restraint_experiments/281/integrator.xml...
************KMSD != 0************
Creating context...
Integrating...
step          0 : time      0.000 ns | lambda   0.000000000000 | protocol_work =        0.000 | RMSD =      0.00026 nm
step       1000 : time      0.004 ns | lambda   0.003996000000 | protocol_work =        0.185 | RMSD =      0.05798 nm
step       2000 : time      0.008 ns | lambda   0.007996000000 | protocol_work =        0.382 | RMSD =      0.08459 nm
step       3000 : time      0.012 ns | lambda   0.011996000000 | protocol_work =        0.520 | RMSD =      0.08785 nm
step       4000 : time      0.016 ns | lambda   0.015996000000 | protocol_work =        0.719 | RMSD =      0.08702 nm
************K_RMSD == 0************
Creating context...
Reading /home/chodera/equilibrated_r

## Run neq with lambda_bonds = 0

In [3]:
test(lambda_param='bonds')

Reading /home/chodera/equilibrated_restraint_experiments/281/281_apo_system.xml...
Reading /home/chodera/equilibrated_restraint_experiments/281/281_apo_state.xml...
Reading /home/chodera/equilibrated_restraint_experiments/281/integrator.xml...
************KMSD != 0************
Creating context...
Integrating...
Changing only one lambda: bonds
step          0 : time      0.000 ns | lambda   0.000000000000 | protocol_work =        0.000 | RMSD =      0.00026 nm
Changing only one lambda: bonds
step       1000 : time      0.004 ns | lambda   0.003996000000 | protocol_work =        0.177 | RMSD =      0.06047 nm
Changing only one lambda: bonds
step       2000 : time      0.008 ns | lambda   0.007996000000 | protocol_work =        0.323 | RMSD =      0.07831 nm
Changing only one lambda: bonds
step       3000 : time      0.012 ns | lambda   0.011996000000 | protocol_work =        0.491 | RMSD =      0.08230 nm
Changing only one lambda: bonds
step       4000 : time      0.016 ns | lambda   0.0

## Run neq with lambda_angles = 0

In [4]:
test(lambda_param='angles')

Reading /home/chodera/equilibrated_restraint_experiments/281/281_apo_system.xml...
Reading /home/chodera/equilibrated_restraint_experiments/281/281_apo_state.xml...
Reading /home/chodera/equilibrated_restraint_experiments/281/integrator.xml...
************KMSD != 0************
Creating context...
Integrating...
Changing only one lambda: angles
step          0 : time      0.000 ns | lambda   0.000000000000 | protocol_work =        0.000 | RMSD =      0.00026 nm
Changing only one lambda: angles
step       1000 : time      0.004 ns | lambda   0.003996000000 | protocol_work =        0.184 | RMSD =      0.06408 nm
Changing only one lambda: angles
step       2000 : time      0.008 ns | lambda   0.007996000000 | protocol_work =        0.282 | RMSD =      0.08663 nm
Changing only one lambda: angles
step       3000 : time      0.012 ns | lambda   0.011996000000 | protocol_work =        0.404 | RMSD =      0.08223 nm
Changing only one lambda: angles
step       4000 : time      0.016 ns | lambda 

## Run neq with lambda_torsions = 0

In [5]:
test(lambda_param='torsions')

Reading /home/chodera/equilibrated_restraint_experiments/281/281_apo_system.xml...
Reading /home/chodera/equilibrated_restraint_experiments/281/281_apo_state.xml...
Reading /home/chodera/equilibrated_restraint_experiments/281/integrator.xml...
************KMSD != 0************
Creating context...
Integrating...
Changing only one lambda: torsions
step          0 : time      0.000 ns | lambda   0.000000000000 | protocol_work =        0.000 | RMSD =      0.00026 nm
Changing only one lambda: torsions
step       1000 : time      0.004 ns | lambda   0.003996000000 | protocol_work =        0.227 | RMSD =      0.06733 nm
Changing only one lambda: torsions
step       2000 : time      0.008 ns | lambda   0.007996000000 | protocol_work =        0.396 | RMSD =      0.07725 nm
Changing only one lambda: torsions
step       3000 : time      0.012 ns | lambda   0.011996000000 | protocol_work =        0.601 | RMSD =      0.08577 nm
Changing only one lambda: torsions
step       4000 : time      0.016 ns

## Run neq with lambda_electrostatics = 0

In [6]:
test(lambda_param='electrostatics')

Reading /home/chodera/equilibrated_restraint_experiments/281/281_apo_system.xml...
Reading /home/chodera/equilibrated_restraint_experiments/281/281_apo_state.xml...
Reading /home/chodera/equilibrated_restraint_experiments/281/integrator.xml...
************KMSD != 0************
Creating context...
Integrating...
Changing only one lambda: electrostatics
step          0 : time      0.000 ns | lambda   0.000000000000 | protocol_work =        0.000 | RMSD =      0.00026 nm
Changing only one lambda: electrostatics
step       1000 : time      0.004 ns | lambda   0.003996000000 | protocol_work =        0.248 | RMSD =      0.06423 nm
Changing only one lambda: electrostatics
step       2000 : time      0.008 ns | lambda   0.007996000000 | protocol_work =        0.506 | RMSD =      0.07163 nm
Changing only one lambda: electrostatics
step       3000 : time      0.012 ns | lambda   0.011996000000 | protocol_work =        0.730 | RMSD =      0.08516 nm
Changing only one lambda: electrostatics
step  

## Run neq with lambda_sterics = 0

In [7]:
test(lambda_param='sterics')

Reading /home/chodera/equilibrated_restraint_experiments/281/281_apo_system.xml...
Reading /home/chodera/equilibrated_restraint_experiments/281/281_apo_state.xml...
Reading /home/chodera/equilibrated_restraint_experiments/281/integrator.xml...
************KMSD != 0************
Creating context...
Integrating...
Changing only one lambda: sterics
step          0 : time      0.000 ns | lambda   0.000000000000 | protocol_work =        0.000 | RMSD =      0.00026 nm
Changing only one lambda: sterics
step       1000 : time      0.004 ns | lambda   0.003996000000 | protocol_work =        0.222 | RMSD =      0.06247 nm
Changing only one lambda: sterics
step       2000 : time      0.008 ns | lambda   0.007996000000 | protocol_work =        0.462 | RMSD =      0.07497 nm
Changing only one lambda: sterics
step       3000 : time      0.012 ns | lambda   0.011996000000 | protocol_work =        0.634 | RMSD =      0.08862 nm
Changing only one lambda: sterics
step       4000 : time      0.016 ns | la